<a href="https://colab.research.google.com/github/vvmnnnkv/private-ai/blob/master/Section%204%20-%20Encrypted%20DB%20Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Encrypted Database Project

Simple key-value DB where data is stored on workers in encrypted form.

In [0]:
!pip install syft

In [2]:
import torch
import syft as sy

hook = sy.TorchHook(torch)

W0711 13:00:02.451985 140630755276672 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0711 13:00:02.472435 140630755276672 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [0]:
class SyDatabase:
  
  def __init__(self, hook):
    num_nodes = 2
    self.nodes = [ sy.VirtualWorker(hook, id="node #%d" % i) for i in range(num_nodes) ]
    self.crypto_provider = sy.VirtualWorker(hook, id="crypto_provider")
    self.db = []
  
  def encrypt(self, key, value = ""):
    key_len = len(key)
    str_data = key + value
    num_data = [key_len] + list(map(ord, list(str_data)))
    tensor = torch.tensor(num_data)
    tensor = tensor.fix_precision().share(*self.nodes, crypto_provider = self.crypto_provider)
    return tensor
  
  def decrypt(self, tensor):
    key_len = int(tensor[0].get().float_precision())
    return ''.join(map(chr, tensor[key_len + 1:].get().float_precision()))
  
  def match_key(self, tensor, key_tensor):
    key_len = int(key_tensor[0].get().float_precision())
    if tensor.shape[0] >= key_len:
      comp = tensor[0:key_len + 1] == key_tensor
      return comp.get().float_precision().sum() == key_len + 1
    return False
  
  def get_index(self, key):
    key_tensor = self.encrypt(key)
    for i, t in enumerate(self.db):
      if self.match_key(t, key_tensor):
        return i
    return None
  
  def get(self, key):
    idx = self.get_index(key)
    if idx != None: return self.decrypt(self.db[idx])
    return None
  
  def set(self, key, value):
    self.db.append(self.encrypt(key, value))
  
  def delete(self, key):
    idx = self.get_index(key)
    if idx != None: del self.db[idx]

In [4]:
db = SyDatabase(hook)
db.set("test", "the secret value")
print("existing value: ", db.get("test"))
print("non-existing value: ", db.get("test-lala"))
db.delete("test-lala")
db.delete("test")
print("deleted value: ", db.get("test"))


existing value:  the secret value
non-existing value:  None
deleted value:  None
